<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/model_selection_aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install optuna

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import tensorflow.keras.optimizers as opt
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import optuna

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
x_train = x_train / 255
x_test = x_test / 255

y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

In [ ]:
def get_model(num_hidden_layers, hidden_layers_units, dropout):
    model = keras.Sequential([Flatten()])
    for i in range(num_hidden_layers):
        model.add(Dense(hidden_layers_units, activation='relu', name=f'Dense_{i}'))
        model.add(Dropout(dropout))

    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-3, 1e-2)
    dropout = trial.suggest_uniform('dropout', 0, 0.5)
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 4)
    hidden_layers_units = trial.suggest_int('hidden_layers_units', 50, 200, 25)

    model = get_model(num_hidden_layers, hidden_layers_units, dropout)

    model.compile(opt.Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    h = model.fit(x=x_train, y=y_train, epochs=100, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)])

    return max(h.history['val_accuracy'])

In [ ]:
study = optuna.create_study(storage='sqlite:///study.db', study_name='MLP', direction='maximize', load_if_exists=True)

[I 2020-09-05 10:44:59,270] Using an existing study with name 'MLP' instead of creating a new one.


In [ ]:
study.optimize(objective, 100, )

1500/1500 [==============================] - 8s 5ms/step - loss: 0.3519 - accuracy: 0.8715 - val_loss: 0.3345 - val_accuracy: 0.8783
Epoch 8/100
 435/1500 [=======>......................] - ETA: 5s - loss: 0.3413 - accuracy: 0.8744


KeyboardInterrupt: ignored